<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Tablets" data-toc-modified-id="Tablets-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tablets</a></span></li><li><span><a href="#Graphemes" data-toc-modified-id="Graphemes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Graphemes</a></span><ul class="toc-item"><li><span><a href="#Primes" data-toc-modified-id="Primes-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Primes</a></span></li><li><span><a href="#Variants" data-toc-modified-id="Variants-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Variants</a></span></li></ul></li></ul></div>

# Checks
Various checks on the correctness of the transformation from ascii transcriptions to a text-fabric data set.

We will perform *grep* commands on the source files, and we will traverse node in Text-Fabric and collect information.

Then we compare these sets of information.

In [1]:
import sys, os, collections, re
from glob import glob
from tf.fabric import Fabric
from utils import Compare

In [2]:
REPO = '~/github/Dans-labs/nino-cunei'
SOURCE = 'uruk'
VERSION = '0.1'
CORPUS = f'{REPO}/tf/{SOURCE}/{VERSION}'
SOURCE_DIR = os.path.expanduser(f'{REPO}/sources/cdli')
TEMP_DIR = os.path.expanduser(f'{REPO}/_temp')

In [3]:
TF = Fabric(locations=[CORPUS], modules=[''], silent=False )

This is Text-Fabric 3.2.0
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

26 features found and 0 ignored


In [4]:
api = TF.load('''
    grapheme prime variant modifier repeat
    damage uncertain remarkable written
    name number badNumbering catalogId period
    srcLn srcLnNum
    op comments
''')
api.makeAvailableIn(globals())
COMP = Compare(api, SOURCE_DIR, TEMP_DIR)

  0.00s loading features ...
   |     0.00s B catalogId            from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.02s B number               from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.06s B grapheme             from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.05s B srcLn                from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.03s B srcLnNum             from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.00s B prime                from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.01s B variant              from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.00s B modifier             from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.01s B repeat               from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.01s B damage               from /Users/dirk/github/Dans-labs/nino-cunei/tf/uruk/0.1
   |     0.00s B unce

## Tablets
We check whether we have the same sequence of tablet numbers.
In TF, the tablet number is stored in the feature `catalogId`.

Note that we also check on the order of the tablets.

In [5]:
def tfTablets():
    tablets = []
    for t in F.otype.s('tablet'):
        (tablet, column, line) = T.sectionFromNode(t)
        tablets.append((F.period.v(t), tablet, F.srcLnNum.v(t), F.catalogId.v(t)))
    return tablets

def grepTablets(gen):
    tablets = []
    prevTablet = None
    for (period, tablet, ln, line) in gen:
        if tablet != prevTablet:
            tablets.append((period, tablet, ln, tablet))
        prevTablet = tablet
    return tablets

In [6]:
COMP.checkSanity(
    grepTablets,
    tfTablets,
)

Number of results: TF 6396; GREP 6396
IDENTICAL: all 6396 items
=    : uruk-iii ◆ P006427 ◆ 1 ◆ P006427
=    : uruk-iii ◆ P006428 ◆ 11 ◆ P006428
=    : uruk-iii ◆ P448701 ◆ 36 ◆ P448701
=    : uruk-iii ◆ P448702 ◆ 50 ◆ P448702
=    : uruk-iii ◆ P448703 ◆ 71 ◆ P448703
=    : uruk-iii ◆ P471695 ◆ 87 ◆ P471695
=    : uruk-iii ◆ P482082 ◆ 114 ◆ P482082
=    : uruk-iii ◆ P482083 ◆ 127 ◆ P482083
=    : uruk-iii ◆ P499393 ◆ 147 ◆ P499393
=    : uruk-iii ◆ P504412 ◆ 166 ◆ P504412
=    : uruk-iii ◆ P504413 ◆ 189 ◆ P504413
=    : uruk-iii ◆ P006438 ◆ 199 ◆ P006438
=    : uruk-iii ◆ P000014 ◆ 220 ◆ P000014
=    : uruk-iii ◆ P000456 ◆ 297 ◆ P000456
=    : uruk-iii ◆ P002718 ◆ 326 ◆ P002718
=    : uruk-iii ◆ P000021 ◆ 341 ◆ P000021
=    : uruk-iii ◆ P000023 ◆ 374 ◆ P000023
=    : uruk-iii ◆ P000025 ◆ 403 ◆ P000025
=    : uruk-iii ◆ P000167 ◆ 500 ◆ P000167
=    : uruk-iii ◆ P000453 ◆ 531 ◆ P000453
=     and 6376 more


## Graphemes

Note that we have defined a function to produce a string value for a full grapheme, including 
repeats, primes, variants and modifiers.
See [utils](utils.py).

A complication is that there are missing line numbers in a few cases, 
so the usual grep pattern does not pick them up.

There a lines that start with `[` and with `|`, so we have to take care we get them.

There are also line numbers with a hyphen in it, such as `6-7`.

In [7]:
lineNumPat = '^(?:(?:[a-zA-Z0-9.\'-]+\s+)|(?=[|\[]))'

### Primes

First an overview of the occurrence of primes.

**N.B.:** This is about primes on *signs*, not on *column* numbers.

In [8]:
for (value, frequency) in F.prime.freqList():
    print(f'{frequency:>5} x {value}')

    9 x 1


Now let us check the primes with grep, directly in the source files.
We look into lines starting with a (hierarchical number), followed by space,
and then later a single of double prime, but not one within a grapheme, such as `GA'AR`.

In [9]:
def tfPrimes():
    primes = []
    for n in F.prime.s(1):
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        primes.append((F.period.v(t), tablet, F.srcLnNum.v(case), f"{COMP.strFromSign(n)}"))
    return primes

In [10]:
def grepPrimes(gen):
    primes = []
    primePat = re.compile(f'{lineNumPat}(.*[\'"][^A].*)')
    graphemePat = re.compile('(?:[0-9]+\([^)]+[\'"]\))|(?:[A-Z0-9~@a-wyz\'-]+\')')
    for (period, tablet, ln, line) in gen:
        match = primePat.match(line)
        if match:
            material = match.group(1)
            graphemes = graphemePat.findall(material)
            for grapheme in graphemes:
                primes.append((period, tablet, ln, grapheme))
    return primes

In [11]:
COMP.checkSanity(
    grepPrimes,
    tfPrimes,
)

Number of results: TF 9; GREP 9
DIFFERENT: first different item is at 1
TF   : uruk-iii ◆ P411604 ◆ 48967 ◆ 1(N24')
GREP : uruk-iii ◆ P411604 ◆ 48967 ◆ 1(N24")
remaining items (TF: 8); GREP: 8
=    : uruk-iii ◆ P411610 ◆ 49069 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49071 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49073 ◆ 1(N24')
=    : uruk-iii ◆ P411610 ◆ 49075 ◆ 1(N24')
=    : uruk-iii ◆ P411539 ◆ 49391 ◆ 1(N24')
=    : uruk-iii ◆ P006437 ◆ 54446 ◆ 1(N30c')
=    : uruk-iii ◆ P464140 ◆ 55938 ◆ 1(N24')
=    : uruk-iii ◆ P464140 ◆ 55939 ◆ 1(N24')
      no more items


This makes it clear: in the transcription there is a strange double prime on the `N(24")`.

### Variants

Overview of variants:

In [12]:
for (value, frequency) in F.variant.freqList():
    print(f'{frequency:>5} x {value}')

23843 x a
 4214 x b
 1534 x c
 1356 x a1
  703 x b1
  194 x a2
  191 x d
  127 x b2
   85 x f
   73 x a3
   40 x e
   29 x c2
   22 x c1
   22 x c3
   14 x c5
   13 x b3
   12 x a0
   12 x d1
   12 x v
   11 x c4
    6 x a4
    6 x g
    5 x d2
    4 x d4
    4 x h
    2 x 3a
    2 x d3
    1 x h2


So there are many variants.

Again, we look for variants in the TF resource and by GREPping them from the sources.

In [13]:
def tfVariants():
    variants = []
    for n in F.otype.s('sign'):
        variant = F.variant.v(n)
        if variant is None:
            continue
        (tablet, column, line) = T.sectionFromNode(n)
        t = L.u(n, otype='tablet')[0]
        case = L.u(n, otype='case')[0]
        
        position = (F.period.v(t), tablet, F.srcLnNum.v(case))
        variants.append((*position, f"{COMP.strFromSign(n)}"))

        written = F.written.v(n)
        if written is not None:
            if '~' in written:
                variants.append((*position, written))

    return variants

There are complications.

#### Order of modifiers and variants
When we extract variants by GREP, we face the problem that the order between modifiers and variants is not consistent.
We see cases of variant and then modifier:

```
3. 1(N14) 8(N01) , RAD~a@g ERIM~a SZU2 A?
```

and cases with modifier and then variant:

```
4. 2(N01) , URUDU@g~b SZU2#
```

both from the same tablet P003407.

We will normalize the order of variant and modifier.

In [14]:
def swap(match):
    return f'{match.group(2)}{match.group(1)}'

def lower(match):
    return f'~{match.group(1).lower()}'

def grepVariants(gen):
    variants = []
    variantPat = re.compile(f'{lineNumPat}(.*[~].*)')
    varPat = re.compile('[~]([A-Z])')
    graphemePat = re.compile('(?:[0-9]+\([^)]+[~][^)]+\))|(?:[A-Z0-9@a-wyz\'-]+[~][0-9@a-wyz]+)')
    combiPat = re.compile('(@[a-ywz0-9]+)(~[a-ywz0-9]+)')
    for (period, tablet, ln, line) in gen:
        match = variantPat.match(line)
        if match:
            material = match.group(1)
            material = varPat.sub(lower, material)
            graphemes = graphemePat.findall(material)
            for grapheme in graphemes:
                grapheme = combiPat.sub(swap, grapheme)
                variants.append((period, tablet, ln, grapheme))
    return variants

In [15]:
COMP.checkSanity(
    grepVariants,
    tfVariants,
)

Number of results: TF 32452; GREP 32452
IDENTICAL: all 32452 items
=    : uruk-iii ◆ P006427 ◆ 8 ◆ SANGA~a
=    : uruk-iii ◆ P006428 ◆ 26 ◆ DUG~b
=    : uruk-iii ◆ P448701 ◆ 42 ◆ AB~a
=    : uruk-iii ◆ P448701 ◆ 42 ◆ APIN~a
=    : uruk-iii ◆ P448701 ◆ 42 ◆ NUN~a
=    : uruk-iii ◆ P448701 ◆ 45 ◆ SZE~a
=    : uruk-iii ◆ P448701 ◆ 45 ◆ NUN~a
=    : uruk-iii ◆ P448702 ◆ 56 ◆ KA~a
=    : uruk-iii ◆ P448702 ◆ 57 ◆ KASZ~b
=    : uruk-iii ◆ P448702 ◆ 57 ◆ NUN~a
=    : uruk-iii ◆ P448702 ◆ 58 ◆ KASZ~a
=    : uruk-iii ◆ P448702 ◆ 59 ◆ 2(N39~a)
=    : uruk-iii ◆ P471695 ◆ 92 ◆ APIN~a
=    : uruk-iii ◆ P471695 ◆ 92 ◆ UR4~a
=    : uruk-iii ◆ P471695 ◆ 93 ◆ EN~a
=    : uruk-iii ◆ P471695 ◆ 94 ◆ BAN~b
=    : uruk-iii ◆ P471695 ◆ 94 ◆ KASZ~c
=    : uruk-iii ◆ P471695 ◆ 97 ◆ PAP~a
=    : uruk-iii ◆ P471695 ◆ 100 ◆ EN~a
=    : uruk-iii ◆ P471695 ◆ 100 ◆ EZINU~d
=     and 32432 more
